In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [ ]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
sys.path.append('../utils')
import Aircraft3dofApproxModel
import Aircraft3dofModel
import FinaltimeFreeCost
import Aircraft3dofConstraints
from scipy.integrate import solve_ivp
from SCP_buffer_deviation import SCP_buffer_deviation
from Scaling import TrajectoryScaling

In [ ]:
ix = 6
iu = 3
tf = 600
N = 100
delT = tf/N
max_iter = 1

In [ ]:
# myModelApprox = Aircraft3dofApproxModel.Aircraft3dofApproxModel('Hello',ix,iu,linearization="numeric_central")
myModelApprox = Aircraft3dofModel.Aircraft3dofContDensity('Hello',ix,iu,linearization="numeric_central")
myModel = Aircraft3dofModel.Aircraft3dof('Hello',ix,iu,linearization="analytic")
myCost = FinaltimeFreeCost.Finaltime('Hello',ix,iu,N)
myConst = Aircraft3dofConstraints.Aircraft3dof('Hello',ix,iu)

x_min = np.zeros(ix)
x_max = np.array([110*1e3,60*1e3,10*1e3,240,np.deg2rad(20),np.pi/2])
u_min = np.array([0,0,0])
u_max = np.array([1.52,np.deg2rad(15),1]) 

myScaling = TrajectoryScaling(x_min,x_max,u_min,u_max,tf)

In [ ]:
xi = np.array([0*1e3,0*1e3,10*1e3,240,0,0])
xf = np.array([110*1e3,-60*1e3,0,95,-np.deg2rad(3),np.deg2rad(80)])
# xf = np.array([110*1e3,-0*1e3,0,95,-np.deg2rad(0),np.deg2rad(0)])

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
ui = np.array([0.0,np.deg2rad(0),0])
uf = np.array([0.0,-np.deg2rad(0),0])

for i in range(N+1) :
    u0[i] = (N-i)/N * ui + i/N * uf
    
# idx_c = int(26 * N / 50)
# for i in range(0,idx_c+1) :
#     u0[i,0] = (idx_c-i)/idx_c * 0.3 + i/idx_c * 0.15
# for i in range(idx_c+1,N+1) :
#     u0[i,0] = 1/2800* (i-idx_c-1)**2 + 0.15
    
# u0[0:10,1] = myConst.phi_max
# u0[-10:,1] = myConst.phi_min
    
    
# u0[0:3,2] = myConst.T_max

# u0[3:70,2] = 0.3

In [ ]:
# w_tr for time is 1
i1 = SCP_buffer_deviation('hello',N,tf,max_iter,myModelApprox,myCost,myConst,myScaling,
          type_discretization="foh",w_c=2,w_bf=1e3,w_tr=1e2,tol_bf=1e-6,tol_tr=1e-6,tol_bc=1)
x,u,xbar,ubar,Tbar,total_num_iter,flag_boundary,l,l_vc,l_tr,x_traj,u_traj  = i1.run(x0,u0,xi,xf)

In [ ]:
print(np.sum(Tbar))

In [ ]:
t_index = [np.sum(Tbar[:i])  for i in range(N+1)]
fig = plt.figure(1,figsize=(15,7))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.plot(x[:, 0], x[:, 1], x[:, 2],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 1], xbar[:, 2],label='solution')
plt.legend()
fig = plt.figure(2,figsize=(15,5))
ax = fig.add_subplot(131)
ax.plot(x[:, 0], x[:, 1],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 1],label='solution')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.legend()
ax = fig.add_subplot(132)
ax.plot(x[:, 1], x[:, 2],label='single shooting')
ax.plot(xbar[:, 1], xbar[:, 2],label='solution')
ax.set_xlabel('Y')
ax.set_ylabel('Z')
ax = fig.add_subplot(133)
ax.plot(x[:, 0], x[:, 2],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 2],label='solution')
ax.set_xlabel('X')
ax.set_ylabel('Z')
plt.legend()
# ax.plot(xbar[:, 0], xbar[:, 1], xbar[:, 2],'o')

In [ ]:
np.abs(x[-1] - xbar[-1])

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(331)
plt.plot(t_index,xbar[:,0],label='solution')
plt.plot(t_index,x[:,0],'--',label='single shooting')
plt.legend(fontsize=15)
plt.title('x (m)')

plt.subplot(332)
plt.plot(t_index,xbar[:,1])
plt.plot(t_index,x[:,1],'--')
plt.title('y (m)')

plt.subplot(333)
plt.plot(t_index,xbar[:,2])
plt.plot(t_index,x[:,2],'--')
plt.title('z (m)')


plt.subplot(334)
plt.plot(t_index,xbar[:,3])
plt.plot(t_index,x[:,3],'--')
plt.plot(t_index,ubar[:,2]*0 + myConst.v_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.v_min,'--',color='tab:red')
plt.title('velocity (m/s)')

plt.subplot(335)
plt.plot(t_index,xbar[:,4])
plt.plot(t_index,x[:,4],'--')
plt.plot(t_index,ubar[:,2]*0 + myConst.gamma_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.gamma_min,'--',color='tab:red')
plt.title('gamma (rad)')

plt.subplot(336)
plt.plot(t_index,xbar[:,5])
plt.plot(t_index,x[:,5],'--')
plt.title('psi (rad)')


t_const_index = np.array([i for i in range(N+1)]) / N * t_index[-1]
plt.subplot(337)
plt.plot(t_index,ubar[:,0],'o-')
# plt.plot(t_index,ubar[:,2]*0 + myConst.CL_max,'--',color='tab:red')
# plt.plot(t_index,ubar[:,2]*0 + myConst.CL_min,'--',color='tab:red')
plt.title('CL')
plt.plot(t_const_index,u0[:,0],'--',label='initial')
plt.legend(fontsize=15)
plt.subplot(338)
plt.plot(t_index,ubar[:,1],'o-')
plt.plot(t_index,ubar[:,2]*0 + myConst.phi_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.phi_min,'--',color='tab:red')
plt.title('phi (rad)')
plt.plot(t_const_index,u0[:,1],'--',label='initial')
plt.subplot(339)
plt.plot(t_index,ubar[:,2],'o-')
plt.plot(t_index,ubar[:,2]*0 + myConst.T_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.T_min,'--',color='tab:red')
plt.title('thrust (N)')
plt.plot(t_const_index,u0[:,2],'--',label='initial')
plt.figure()
plt.plot(t_index,np.array(t_index)*0,'o-')

In [ ]:
for ubar_ in u_traj[0:5] :
    plt.plot(t_index,ubar_[:,0],alpha=0.5,color='tab:blue')
# plt.title('CL')

In [ ]:
T = 15.04 - 0.00649 * xbar[:,2] # celsius
p = 101.29 * np.power((T+273.1)/288.08,5.256)
rho = p / (0.2869 * (T + 273.1))

F_lift = 0.5 * rho * xbar[:,3] * xbar[:,3] * myModel.Sw * ubar[:,0]
F_draf = 0.5 * rho * xbar[:,3] * xbar[:,3] * myModel.Sw * (myModel.CD0 + myModel.K  * ubar[:,0] * ubar[:,0])

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(t_index,rho)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(338)
plt.plot(t_index,F_lift)
plt.title('Lift force (N)')

plt.subplot(339)
plt.plot(t_index,F_draf)
plt.title('Drag force (N)')

In [ ]:
a = np.random.randn(3,2,3)
print(a)
print()
b = np.array([1,2,3])
print((a.T*b).T)